In [1]:
import os
from dotenv import find_dotenv, load_dotenv
from fredapi import Fred
import pandas as pd
import pandas_datareader as pdr
import sqlite3
import logging

%matplotlib inline
import matplotlib.pyplot as plt

load_dotenv(find_dotenv())
fred_api_key = os.environ.get('FRED_API_KEY')
fred = Fred(api_key=fred_api_key)

In [2]:
ticker = 'CYS'
series_ids = ['SP500', 'DGS10', 'DGS5', 'USD3MTD156N', 'USD1WKD156N', 'FF']

In [3]:
conn = sqlite3.connect(os.environ.get('DB_PATH'))


In [4]:
for id in series_ids:
    df = fred.get_series_info(id).to_frame().T
    try:
        df.to_sql('data_series', conn, if_exists='append', index=False)
    except sqlite3.IntegrityError as e:
        logging.warning(e)
        continue

In [5]:
for row in conn.execute('select id, title from data_series'):
    print(row)

('FF', 'Effective Federal Funds Rate')
('SP500', 'S&P 500')
('DGS10', '10-Year Treasury Constant Maturity Rate')
('DGS5', '5-Year Treasury Constant Maturity Rate')
('USD3MTD156N', '3-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar')
('USD1WKD156N', '1-Week London Interbank Offered Rate (LIBOR), based on U.S. Dollar')


In [6]:
for id in series_ids:
    q = conn.execute("SELECT value FROM data where ds_id=?", [str(id)])
    if q.fetchone() is not None:
        print('{0} is already in the DB!'.format(id))
    else:
        print('{0} not found...'.format(id))

SP500 is already in the DB!
DGS10 is already in the DB!
DGS5 is already in the DB!
USD3MTD156N is already in the DB!
USD1WKD156N is already in the DB!
FF is already in the DB!


In [10]:
for id in series_ids:
    q_test = conn.execute("SELECT value FROM data where ds_id=?", [str(id)])
    if q.fetchone() is not None:
        print('{0} is already in the DB!'.format(id))
    else:
        query = conn.execute("SELECT observation_start FROM data_series WHERE id=?", [str(id)])
        data = fred.get_series(id, observation_start=query.fetchone()[0])
        df = data.to_frame(name='value').rename(columns={'index':'date'}, inplace=True)
        df['ds_id'] = id
        df.dropna(inplace=True)
        df.to_sql('data', conn, if_exists='append', index=True, index_label='date')

SP500 is already in the DB!
DGS10 is already in the DB!
DGS5 is already in the DB!
USD3MTD156N is already in the DB!
USD1WKD156N is already in the DB!
FF is already in the DB!


In [12]:
for id in series_ids:
    q = conn.execute('SELECT * from data where ds_id=? LIMIT 5', [str(id)])
    print(q.fetchall())

[(1, '2007-12-05 00:00:00', 1485.01, 'SP500'), (2, '2007-12-07 00:00:00', 1504.66, 'SP500'), (3, '2007-12-10 00:00:00', 1515.96, 'SP500'), (4, '2007-12-11 00:00:00', 1477.65, 'SP500'), (5, '2007-12-12 00:00:00', 1486.59, 'SP500')]
[(2521, '1962-01-02 00:00:00', 4.06, 'DGS10'), (2522, '1962-01-03 00:00:00', 4.03, 'DGS10'), (2523, '1962-01-04 00:00:00', 3.99, 'DGS10'), (2524, '1962-01-05 00:00:00', 4.02, 'DGS10'), (2525, '1962-01-08 00:00:00', 4.03, 'DGS10')]
[(16490, '1962-01-02 00:00:00', 3.88, 'DGS5'), (16491, '1962-01-03 00:00:00', 3.87, 'DGS5'), (16492, '1962-01-04 00:00:00', 3.86, 'DGS5'), (16493, '1962-01-05 00:00:00', 3.89, 'DGS5'), (16494, '1962-01-08 00:00:00', 3.91, 'DGS5')]
[(30459, '1986-01-02 00:00:00', 8, 'USD3MTD156N'), (30460, '1986-01-03 00:00:00', 8.0625, 'USD3MTD156N'), (30461, '1986-01-06 00:00:00', 8.125, 'USD3MTD156N'), (30462, '1986-01-07 00:00:00', 8.125, 'USD3MTD156N'), (30463, '1986-01-08 00:00:00', 8, 'USD3MTD156N')]
[(38526, '1997-12-01 00:00:00', 5.69531, 'U